In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

In [3]:
docs_path = "../data/raw/"
documents = []

document_infos = []

for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        with open(os.path.join(docs_path, filename), "r", encoding="utf-8") as f:
            text = f.read()
            documents.append(text)
            
            metadata = {
                "filename": filename,
                "type": "cv" if "cv" in filename.lower() else "project" if "project" in filename.lower() else "portfolio_section"
            }
            document_infos.append(metadata)

In [4]:
document_infos

[{'filename': 'about_me_portfolio.txt', 'type': 'portfolio_section'},
 {'filename': 'bachelor_project.txt', 'type': 'project'},
 {'filename': 'cv.txt', 'type': 'cv'},
 {'filename': 'diablo_project.txt', 'type': 'project'},
 {'filename': 'education_portfolio.txt', 'type': 'portfolio_section'},
 {'filename': 'experience_portfolio.txt', 'type': 'portfolio_section'},
 {'filename': 'facial_recognition_project.txt', 'type': 'project'},
 {'filename': 'skills_portfolio.txt', 'type': 'portfolio_section'},
 {'filename': 'smart_chess_project.txt', 'type': 'project'},
 {'filename': 'splash_project.txt', 'type': 'project'},
 {'filename': 'sweepresort_project.txt', 'type': 'project'},
 {'filename': 'taskify_project.txt', 'type': 'project'},
 {'filename': 'vclinic_project.txt', 'type': 'project'},
 {'filename': 'zaki_project.txt', 'type': 'project'},
 {'filename': 'zombie_invasion_project.txt', 'type': 'project'}]

In [5]:
documents

["Hi, I'm Ahmed\nSoftware Engineer\n\nI'm a software engineer with a strong background in building scalable applications.\nMy experience includes both frontend and backend, working with modern JavaScript frameworks. \nI focus on delivering clean, efficient code and solving real world problems.",
 'Title: Breast Cancer Diagnosis System\nCompany: Bachelor Project\nGitHub: https://github.com/Ahmedmk11/breast-cancer-diagnosis\nLink: https://doi.org/10.13140/RG.2.2.28574.24644\nDescription: An embedded system for real-time breast cancer diagnosis using ultrasound imaging and a multi-stage CNN approach.\nTools: Python, PyTorch, TensorFlow, Scikit-learn, Pandas, NumPy, Transfer Learning, ResNet-101, InceptionV3, Raspberry Pi 3, Bioinformatics, Computer Vision, Deep Learning, Embedded Systems\nSignificance: 1\nCompleted: True\n',
 'Ahmed Mahmoud\nCairo, Egypt | ahmedmahmoudkzm@gmail.com | +20 1550800848\nhttps://www.ahmedmahmoud.dev/\nhttps://www.linkedin.com/in/ahmed-mahmoud-350b21214/\nhttps

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs_chunks = []
metadatas = []

for doc, meta in zip(documents, document_infos):
    chunks = text_splitter.split_text(doc)
    for chunk in chunks:
        docs_chunks.append(chunk)
        metadatas.append(meta)

In [8]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\pc\Desktop\Work\Personal\personal-chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pc\Desktop\Work\Personal\personal-chatbot\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run

In [9]:
embeddings_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
vectorstore = FAISS.from_texts(docs_chunks, embeddings_model, metadatas=metadatas)

In [11]:
vectorstore.save_local("../data/vectorstore")

print("Vector store created and saved in ../data/vectorstore")

Vector store created and saved in ../data/vectorstore
